---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# https://eagronin.github.io/university-towns-analyze/
# http://modasserbillah.ml/2018-07-01-hypothesis-testing/

# Assignment 4 - Hypothesis Testing

This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {"OH": "Ohio",
          "KY": "Kentucky",
          "AS": "American Samoa",
          "NV": "Nevada",
          "WY": "Wyoming",
          "NA": "National",
          "AL": "Alabama",
          "MD": "Maryland",
          "AK": "Alaska",
          "UT": "Utah",
          "OR": "Oregon",
          "MT": "Montana",
          "IL": "Illinois",
          "TN": "Tennessee",
          "DC": "District of Columbia",
          "VT": "Vermont",
          "ID": "Idaho",
          "AR": "Arkansas",
          "ME": "Maine",
          "WA": "Washington",
          "HI": "Hawaii",
          "WI": "Wisconsin",
          "MI": "Michigan",
          "IN": "Indiana",
          "NJ": "New Jersey",
          "AZ": "Arizona",
          "GU": "Guam",
          "MS": "Mississippi",
          "PR": "Puerto Rico",
          "NC": "North Carolina",
          "TX": "Texas",
          "SD": "South Dakota",
          "MP": "Northern Mariana Islands",
          "IA": "Iowa",
          "MO": "Missouri",
          "CT": "Connecticut",
          "WV": "West Virginia",
          "SC": "South Carolina",
          "LA": "Louisiana",
          "KS": "Kansas",
          "NY": "New York",
          "NE": "Nebraska",
          "OK": "Oklahoma",
          "FL": "Florida",
          "CA": "California",
          "CO": "Colorado",
          "PA": "Pennsylvania",
          "DE": "Delaware",
          "NM": "New Mexico",
          "RI": "Rhode Island",
          "MN": "Minnesota",
          "VI": "Virgin Islands",
          "NH": "New Hampshire",
          "MA": "Massachusetts",
          "GA": "Georgia",
          "ND": "North Dakota",
          "VA": "Virginia"}

In [131]:
def get_list_of_university_towns():
    """Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'."""
    
    # Read the data
    df = pd.read_table("university_towns.txt", header = None)
    
    # Create a new list to store the data
    university_town_list = []
    
    # Loop through each row, clean the extra text, and map each state and region
    current_state = None
    for index, row in df.iterrows():
        # If this is a region row
        if "edit" not in row[0]:
            # Append state and region
            university_town_list.append([current_state, row[0].split("(")[0].strip()])
        # If this is state row
        else:
            current_state = row[0].split("[")[0].strip()
            
    university_town_df = pd.DataFrame(university_town_list, columns = ["State", "RegionName"])
    
    return university_town_df

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [99]:
# def get_list_of_university_towns():
#     """Returns a DataFrame of towns and the states they are in from the 
#     university_towns.txt list. The format of the DataFrame should be:
#     DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
#     columns=["State", "RegionName"]  )
    
#     The following cleaning needs to be done:

#     1. For "State", removing characters from "[" to the end.
#     2. For "RegionName", when applicable, removing every character from " (" to the end.
#     3. Depending on how you read the data, you may need to remove newline character '\n'."""
    
#     # Read the data
#     df = pd.read_table("university_towns.txt", header = None)
    
#     # Create a new list to store the data
#     university_town_list = []
    
#     # Loop through each row, clean the extra text, and map each state and region
#     current_state = None
#     for index, row in df.iterrows():
#         # If this is a region row
#         if ("(" in row[0] and
#             ")" in row[0] and
#             current_state is not None):
#             # Append state and region
#             university_town_list.append([current_state, row[0].split("(")[0]])
#         # If this is state row
#         else:
#             current_state = row[0].split("[")[0]
            
#     university_town_df = pd.DataFrame(university_town_list, columns = ["State", "RegionName"])
    
#     return university_town_df

# get_list_of_university_towns()

# def get_list_of_university_towns():
#     """Returns a DataFrame of towns and the states they are in from the 
#     university_towns.txt list. The format of the DataFrame should be:
#     DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
#     columns=["State", "RegionName"]  )
    
#     The following cleaning needs to be done:

#     1. For "State", removing characters from "[" to the end.
#     2. For "RegionName", when applicable, removing every character from " (" to the end.
#     3. Depending on how you read the data, you may need to remove newline character '\n'."""
    
#     # Read the data
#     df = pd.read_table("university_towns.txt", header = None)
    
#     # Create a new list to store the data
#     university_town_list = []
    
#     # Loop through each row, clean the extra text, and map each state and region
#     current_state = None
#     for index, row in df.iterrows():
#         # If this is a region row
#         if "edit" not in row[0]:
#             # Append state and region
#             university_town_list.append([current_state, row[0].split("(")[0]])
#         # If this is state row
#         else:
#             current_state = row[0].split("[")[0]
            
#     university_town_df = pd.DataFrame(university_town_list, columns = ["State", "RegionName"])
    
#     return university_town_df

# get_list_of_university_towns()

# def get_list_of_university_towns():
#     """Returns a DataFrame of towns and the states they are in from the 
#     university_towns.txt list. The format of the DataFrame should be:
#     DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
#     columns=["State", "RegionName"]  )
    
#     The following cleaning needs to be done:

#     1. For "State", removing characters from "[" to the end.
#     2. For "RegionName", when applicable, removing every character from " (" to the end.
#     3. Depending on how you read the data, you may need to remove newline character '\n'."""
    
#     # Read the data
#     university_town_df = pd.read_table("university_towns.txt", header = None)
#     university_town_df.columns = ["RegionName"]
    
#     # Create column to map university town and states
#     university_town_df["State"] = np.nan
#     # Loop through the data
#     for town in university_town_df["RegionName"]:
#         if "[edit]" in town:
#             university_town_df.State[university_town_df.RegionName == town] = town
            
#     # Fill empty entry
#     university_town_df["State"] = university_town_df["State"].fillna(method = "ffill")
#     # Filter out the state entry
#     university_town_df = university_town_df[university_town_df.RegionName != university_town_df.State]
#     # Reset the index and do not add extra
#     university_town_df = university_town_df.reset_index(drop = True)
    
#     # Remove extra text for region name
#     for town in university_town_df.RegionName:
#         town_clean = town.split("(")[0]
#         university_town_df.RegionName[university_town_df.RegionName == town] = town_clean
        
#     # Remove extra text for state
#     for state in university_town_df.State:
#         state_clean = state.split("[")[0]
#         university_town_df.State[university_town_df.State == state] = state_clean
        
#     # Set the column name
#     column_name = ["State", "RegionName"]
#     university_town_df = university_town_df[column_name]
    
#     return university_town_df

# get_list_of_university_towns()

In [96]:
def get_gdp_data():
    # Read data
    df = pd.read_excel("gdplev.xls", skiprows = 219)
    
    # Drop the empty rows
    df = df.dropna(axis = "columns", how = "all")
    
    # Rename the columns
#     df.columns = ["Year",
#                   "Y GDP Billion Current Dollar",
#                   "Y GDP Billion Chained 2009 Dollar",
#                   "Quarter",
#                   "Q GDP Billion Current Dollar",
#                   "Q GDP Billion Chained 2009 Dollar"]
    
    df.columns = ["Quarter",
                  "Q GDP Billion Current Dollar",
                  "Q GDP Billion Chained 2009 Dollar"]
    
    # Use the chained 2009 dollar
    #df.drop(columns = ["Q GDP Billion Current Dollar"], inplace = True)
    
    # Do not need first 3 column
    #df.drop(columns = ["Year", "Y GDP Billion Current Dollar", "Y GDP Billion Chained 2009 Dollar"], inplace = True)
    
    return df

In [97]:
def get_recession_start():
    """Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3"""
    
    # Get the data
    df = get_gdp_data()
    
    # List for recession quarter
    recession_start_quarter = []
    
    # Loop through and check for recession
    for i in range(len(df.index)):
        # Check if we are at the end
        if i <= len(df.index) - 5:
            # Get 5 quarter of data
            quarter_1 = df.iloc[i][2]
            quarter_2 = df.iloc[i + 1][2]
            quarter_3 = df.iloc[i + 2][2]
            quarter_4 = df.iloc[i + 3][2]
            quarter_5 = df.iloc[i + 4][2]
            
            # Check if it is a recession
            if (quarter_1 > quarter_2 and # Decline
                quarter_2 > quarter_3 and # Decline
                quarter_3 < quarter_4 and # Growth
                quarter_4 < quarter_5):   # Growth
                # Add the recession quarter start
                # The correct quarter of recession start is apparently 1 quarter prior to recession
                recession_start_quarter.append(df.iloc[i - 1][0])
    
    return recession_start_quarter[0]

get_recession_start()

'2008q3'

In [98]:
def get_recession_end():
    """Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3"""
    
    # Get the data
    df = get_gdp_data()
    
    # List for recession quarter
    recession_end_quarter = []
    
    # Loop through and check for recession
    for i in range(len(df.index)):
        # Check if we are at the end
        if i <= len(df.index) - 5:
            # Get 5 quarter of data
            quarter_1 = df.iloc[i][1]
            quarter_2 = df.iloc[i + 1][1]
            quarter_3 = df.iloc[i + 2][1]
            quarter_4 = df.iloc[i + 3][1]
            quarter_5 = df.iloc[i + 4][1]
            
            # Check if it is a recession
            if (quarter_1 > quarter_2 and # Decline
                quarter_2 > quarter_3 and # Decline
                quarter_3 < quarter_4 and # Growth
                quarter_4 < quarter_5):   # Growth
                # Add the recession quarter start
                recession_end_quarter.append(df.iloc[i + 4][0])
    
    return recession_end_quarter[0]

get_recession_end()

'2009q4'

In [92]:
def get_recession_bottom():
    """Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3"""
    
    # Get the data
    df = get_gdp_data()
    
    # Get the start and end of the recession
    recession_start = df.index[df["Quarter"] == get_recession_start()][0]
    recession_end = df.index[df["Quarter"] == get_recession_end()][0]
    
    # Calculate the bottom
    result = df.iloc[df.iloc[recession_start:recession_end + 1, 1].idxmin(), 0]
    
    return result

get_recession_bottom()

'2009q2'

In [93]:
def get_house_data():
    # Read the data
    df = pd.read_csv("City_Zhvi_AllHomes.csv", header = 0)
    
    # Drop the columns not needed
    df = df.drop(["Metro", "CountyName", "SizeRank"], axis = 1)
    
    return df

In [94]:
def convert_housing_data_to_quarters():
    """Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    """
    
    # Get the data
    house_data = get_house_data()
    
    # Remove duplicate
    df = house_data.drop_duplicates(subset = "RegionID", keep = "first")
    
    # Reshape row
    # Set the index
    house_data = house_data.set_index(["State", "RegionName", "RegionID"], drop = True)
    # Set index header name
    house_data.columns.name = "Month"
    # Reshape the data frame
    df = house_data.stack()
    # Name the new data frame
    df.name = "Value"
    # Reset the index
    df = df.reset_index()
    # Convert column type
    df.Month = pd.to_datetime(df.Month)
    # Get the year and quarter
    df["Year"] = df.Month.dt.year
    df["Quarter"] = df.Month.dt.quarter
    
    # Group data by each quarter
    df = df.groupby(["State",
                     "RegionName",
                     "RegionID",
                     "Year",
                     "Quarter"]).mean()["Value"]
    # Reset the index
    df = df.reset_index()
    # Create the quarter "Date"
    df["Date"] = df.Year.apply(str) + "q" + df.Quarter.apply(str)
    
    # Do not need anything prior to 2000
    df = df[df.Year >= 2000]
    
    # Reshape column
    # Drop the "Year" and "Quarter" column
    df = df.drop(["Year", "Quarter"], axis = 1)
    # Set the index
    df = df.set_index(["State",
                       "RegionName",
                       "RegionID",
                       "Date"])
    # Name the data frame
    df = df["Value"]
    # Unstack base on the "Date"
    df = df.unstack("Date")
    # Reset the index
    df = df.reset_index()
    # Rename the column "State" to "Code"
    df.rename(columns = {"State": "Code"}, inplace = True)
    
    # Merge house data with state data
    # Create data frame base on state data
    state_code_df = pd.DataFrame.from_dict(states, orient = "index")
    # Reset the index
    state_code_df = state_code_df.reset_index()
    # Set the column
    state_code_df.columns = ["Code", "State"]
    # Merge the data
    df = df.merge(state_code_df, how = "inner", on = "Code")
    # Do not need the state "Code" and "RegionID" column, drop it
    df = df.drop("Code", axis = 1)
    df = df.drop("RegionID", axis = 1)
    # Set the new index
    df = df.set_index(["State", "RegionName"])
    
    return df

convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State   RegionName                                                  
Alaska  Anchor Point            NaN            NaN            NaN   
        Anchorage     174633.333333  175266.666667  179566.666667   
        Fairbanks     163200.000000  165033.333333  169300.000000   
        Homer                   NaN            NaN            NaN   
        Juneau        192466.666667  194300.000000  195166.666667   
...                             ...            ...            ...   
Wyoming Burns         101533.333333  104566.666667  108366.666667   
        Casper         89233.333333   89600.000000   89733.333333   
        Cheyenne      116866.666667  120033.333333  121533.333333   
        Evansville    128033.333333  128766.666667  130833.333333   
        Pine Bluffs    93733.333333   95066.666667   94633.333333   

                             2000q4         2001q1         2001q2  \
State   RegionName                                                  
Alaska  Anchor Point            NaN            NaN            NaN   
        Anchorage     182833.333333  182766.666667  183933.333333   
        Fairbanks     172800.000000  164433.333333  157800.000000   
        Homer                   NaN            NaN            NaN   
        Juneau        194166.666667  197166.666667  199533.333333   
...                             ...            ...            ...   
Wyoming Burns         113000.000000  115833.333333  117200.000000   
        Casper         93166.666667   95500.000000   97633.333333   
        Cheyenne      123633.333333  125533.333333  126300.000000   
        Evansville    132066.666667  130566.666667  131433.333333   
        Pine Bluffs    98066.666667  103233.333333  104600.000000   

                             2001q3         2001q4         2002q1  \
State   RegionName                                                  
Alaska  Anchor Point            NaN            NaN            NaN   
        Anchorage     188566.666667  191866.666667  193966.666667   
        Fairbanks     158200.000000  154666.666667  152766.666667   
        Homer                   NaN            NaN            NaN   
        Juneau        202433.333333  206300.000000  209066.666667   
...                             ...            ...            ...   
Wyoming Burns         117800.000000  117633.333333  117333.333333   
        Casper         99433.333333  100633.333333  101733.333333   
        Cheyenne      126466.666667  128133.333333  128466.666667   
        Evansville    132400.000000  133466.666667  133300.000000   
        Pine Bluffs   106500.000000  104066.666667  102233.333333   

                             2002q2  ...         2014q2         2014q3  \
State   RegionName                   ...                                 
Alaska  Anchor Point            NaN  ...  157800.000000  158000.000000   
        Anchorage     196700.000000  ...  278200.000000  280766.666667   
        Fairbanks     154533.333333  ...  209233.333333  207100.000000   
        Homer                   NaN  ...  233033.333333  232900.000000   
        Juneau        209866.666667  ...  328266.666667  331833.333333   
...                             ...  ...            ...            ...   
Wyoming Burns         117233.333333  ...  168866.666667  161933.333333   
        Casper        101533.333333  ...  175766.666667  177300.000000   
        Cheyenne      129633.333333  ...  177466.666667  176733.333333   
        Evansville    131066.666667  ...  296733.333333  305666.666667   
        Pine Bluffs   103566.666667  ...  148666.666667  154366.666667   

                             2014q4         2015q1         2015q2  \
State   RegionName                                                  
Alaska  Anchor Point  155566.666667  154266.666667  154100.000000   
        Anchorage     281700.000000  284166.666667  287166.666667   
        Fairbanks     207000.000000  207766.666667  206466.666667   
        Homer         23106

In [132]:
def run_ttest():
    """First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss)."""
    
    # Get the recession start
    recession_start = get_recession_start()
    # Get the recession bottom
    recession_bottom = get_recession_bottom()
    # Get house data
    house_df = convert_housing_data_to_quarters()
    # Only need the columns from recession start to bottom
    house_df = house_df.loc[:, recession_start:recession_bottom]
    # Reset the index for the data
    house_df.reset_index(inplace = True)
    # Calculate the ratio between recession start and end, add it to new column
    house_df["PriceRatio"] = (house_df[recession_start] - house_df[recession_bottom]) / house_df[recession_start]
    
    # Get the university town data for split
    university_town_df = get_list_of_university_towns()
    # Extract the region name to a list
    university_town_list = university_town_df["RegionName"].tolist()
    # Add a condition column for split
    house_df["UniversityTown"] = house_df.RegionName.apply(lambda x: x in university_town_list)
    
    # Split the data
    university_df = house_df[house_df.UniversityTown].copy().dropna()
    non_university_df = house_df[~house_df.UniversityTown].copy().dropna()
    
    # Get the p-value by applying t-test on the split data ratio
    p = ttest_ind(university_df["PriceRatio"], non_university_df["PriceRatio"])[1]
    
    # Check whether or not to reject the null hypothesis
    different = p < 0.01
    
    # Check which town have lower house price ratio (mean) during recession
    lower = "university town" if university_df["PriceRatio"].mean() < non_university_df["PriceRatio"].mean() else "non-university town"
    
    return (different, p, lower)

run_ttest()

(True, 0.0003664160159552616, 'university town')